### Previous: <a href = "tensorflow_01.ipynb">2.1 Structure</a>

# <center> TensorFlow </center>
## <center>2.2 Weights</center>


# Explanation

The specification of what a layer does to its input data is stored in the layer’s
__weights__, which in essence are a bunch of numbers. In technical terms, you would say
that the transformation implemented by a layer is "parametrized" by its weights.
In fact, weights are also sometimes called the "parameters" of a layer. <br>
In this context, "learning" will mean finding a set of values for the weights of all layers in a network, such that the
network will correctly map your example inputs to their associated targets. But here’s the
thing: a deep neural network can contain tens of millions of parameters. Finding the
correct value for all of them may seem like a daunting task, especially since modifying
the value of one parameter will affect the behavior of all others!
<br>
<img src="img/weights.jpg" width="50%" /><br>


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import matplotlib.pyplot as plt
import numpy as np
import random as ran

def TRAIN_SIZE(num):
    #print ('Total Training Images in Dataset = ' + str(mnist.train.images.shape))
    #print ('--------------------------------------------------')
    x_train = mnist.train.images[:num,:]
    #print ('x_train Examples Loaded = ' + str(x_train.shape))
    y_train = mnist.train.labels[:num,:]
    #print ('y_train Examples Loaded = ' + str(y_train.shape))
    #print('')
    return x_train, y_train

def TEST_SIZE(num):
    #print ('Total Test Examples in Dataset = ' + str(mnist.test.images.shape))
    #print ('--------------------------------------------------')
    x_test = mnist.test.images[:num,:]
    #print ('x_test Examples Loaded = ' + str(x_test.shape))
    y_test = mnist.test.labels[:num,:]
    #print ('y_test Examples Loaded = ' + str(y_test.shape))
    return x_test, y_test

def display_digit(num):
    print(y_train[num])
    label = y_train[num].argmax(axis=0)
    image = x_train[num].reshape([28,28])
    plt.title('Example: %d  Label: %d' % (num, label))
    plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    plt.show()

def display_mult_flat(start, stop):
    images = x_train[start].reshape([1,784])
    for i in range(start+1,stop):
        images = np.concatenate((images, x_train[i].reshape([1,784])))
    plt.imshow(images, cmap=plt.get_cmap('gray_r'))
    plt.show()
    
x_train, y_train = TRAIN_SIZE(55000)

import tensorflow as tf
sess = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Example

In the next step, we will define the weights W and bias b. These two values are the grunt workers of the classifier. They will be the only values we will need to calculate our prediction after the classifier is trained.

We will first set our weight and bias values to zeros because TensorFlow will optimize these values later. Notice how our W is a collection of 784 values for each of the 10 classes.

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

You can think of the weights as 10 cheat sheets for each number, which is similar to how a teacher uses a cheat sheet transparency to grade a multiple choice exam.
<br> <br>The bias is a little beyond the scope of this workshop, but you can think of it as a special relationship with the weights that influences the final answer.
<br> <br>
We will now define y, which is our classifier function.  We make our prediction by multiplying each flattened digit by our weight and then adding our bias. __Matmul__ is the function for multiplying matrices.

In [ ]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

## in a graph:
<img src="https://www.tensorflow.org/versions/master/images/softmax-regression-scalargraph.png" width = "60%"/>

## in a vector equation:
<img src="https://www.tensorflow.org/versions/master/images/softmax-regression-vectorequation.png" width = "60%"/>

In [ ]:
print(y)

In [ ]:
x_train, y_train = TRAIN_SIZE(3)
sess.run(tf.global_variables_initializer())
sess.run(y, feed_dict={x: x_train})
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
x_train, y_train = TRAIN_SIZE(5500)
x_test, y_test = TEST_SIZE(10000)
LEARNING_RATE = 0.1
TRAIN_STEPS = 2500
init = tf.global_variables_initializer()
sess.run(init)
training = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
for i in range(TRAIN_STEPS+1):
    sess.run(training, feed_dict={x: x_train, y_: y_train})
    if i%100 == 0:
        print('Training Step:' + str(i) + '  Accuracy =  ' + str(sess.run(accuracy, feed_dict={x: x_test, y_: y_test})) + '  Loss = ' + str(sess.run(cross_entropy, {x: x_train, y_: y_train})))

# Weights
<img src="img/Weights.png" width = "50%"/><br>
In this image, if the red pixel is filled, the digit being classified is very likely to be a one, thus has positive weight. <br>
On the right side, if the blue pixel is filled, the digit being classified is very likely not a zero, thus has negative weight.  
<br>
After we have calculated our weight cheatsheet, we can create a graph with the following code:

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i+1)
    weight = sess.run(W)[:,i]
    plt.title(i)
    plt.imshow(weight.reshape([28,28]), cmap=plt.get_cmap('seismic'))
    frame1 = plt.gca()
    frame1.axes.get_xaxis().set_visible(False)
    frame1.axes.get_yaxis().set_visible(False)
plt.show()

This is a visualization of our weights from 0-9, which is the most important aspect of our classifier. The bulk of the work of machine learning is figuring out what the optimal weights are. Once they are calculated, you have the “cheat sheet” and can easily find answers. 

This is why neural networks can be easily used within mobile devices, as  the model, once trained, doesn’t take up that much room to store or computing power to calculate. 

Our classifier makes its prediction by comparing how similar or different the digit is to the red and blue. <br>You can think the darker the red, the better of a hit; white as neutral; and blue as misses.

So, now that we have our cheat sheet, let’s load one example and apply our classifier to that one example.

In [ ]:
x_train, y_train = TRAIN_SIZE(1)
display_digit(0)

Let’s look at our predictor y. 
This gives us a (1x10) matrix with each column containing one probability.

In [ ]:
answer = sess.run(y, feed_dict={x: x_train})
print(answer)

This is not very useful for us. So, we use the __argmax__ function to return the position of the highest value and that gives us our prediction.

In [ ]:
answer.argmax()

Let's create a function to make predictions on a random digit in this data set.

In [ ]:
def display_compare(num):
    # THIS WILL LOAD ONE TRAINING EXAMPLE
    x_train = mnist.train.images[num,:].reshape(1,784)
    y_train = mnist.train.labels[num,:]
    # THIS GETS OUR LABEL AS A INTEGER
    label = y_train.argmax()
    # THIS GETS OUR PREDICTION AS A INTEGER
    prediction = sess.run(y, feed_dict={x: x_train}).argmax()
    plt.title('Prediction: %d Label: %d' % (prediction, label))
    plt.imshow(x_train.reshape([28,28]), cmap=plt.get_cmap('gray_r'))
    plt.show()

Now, let's try out the function. 

In [ ]:
display_compare(ran.randint(0, 55000))

In [ ]:
display_compare(2)

Here, the digit has been predicted incorrectly.  

Why do you think it got it wrong?

Notice what happens to the visualizations of the weights when you use 1-10 training examples. It becomes clear that using too little data makes it very hard to generalize. Here is an animation showing how the weights change as you increase your training size. 

<img src="img/a.gif" width = "65%" />

# Feedback